<a href="https://colab.research.google.com/github/anhnguyenvv/AI-Lab1-Searching/blob/main/ai_ali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio -q
!pip install openai -q
!pip install gtts -q
!pip install pydub -q
!pip install config -q
!pip install git+https://github.com/openai/whisper.git -q
!sudo apt update && sudo apt install ffmpeg
!pip install gdown -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.2 MB/s eta 

In [2]:
import gradio as gr
import config
from gtts import gTTS
import os
import subprocess
from pydub import AudioSegment
import math
import whisper
from openai import OpenAI
import requests, gdown
import datetime

### load whisper model

In [3]:
import torch

# Checking if NVIDIA GPU is available
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
W_model = whisper.load_model("medium", device=DEVICE)

100%|█████████████████████████████████████| 1.42G/1.42G [00:22<00:00, 68.1MiB/s]


In [4]:
#File Project1_Data.json
file_id = ["1jpM-ZP0h6OXDyuAfN1V6U7fvkdZuLLpU",
           "1uwpA8Wuix2QJ0gvvnZk7CtaSsoC21iEj",
           "1iLKcK5Z9IaYrKzaPQLMIZm3vJ7DssCAz"
]
import os
os.makedirs('Samples')
datafilename =[ "./Samples/Sample1.mp3",
        "./Samples/Sample2.mp3",
        "./Samples/Sample3.mp3"]
for i in range(len(file_id)):
   with open(datafilename[i], 'wb') as f:
     gdown.download(f"https://drive.google.com/uc?id={file_id[i]}", datafilename[i])

Downloading...
From: https://drive.google.com/uc?id=1jpM-ZP0h6OXDyuAfN1V6U7fvkdZuLLpU
To: /content/Samples/Sample1.mp3
100%|██████████| 7.69M/7.69M [00:00<00:00, 25.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uwpA8Wuix2QJ0gvvnZk7CtaSsoC21iEj
To: /content/Samples/Sample2.mp3
100%|██████████| 8.13M/8.13M [00:00<00:00, 25.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1iLKcK5Z9IaYrKzaPQLMIZm3vJ7DssCAz
To: /content/Samples/Sample3.mp3
100%|██████████| 10.1M/10.1M [00:00<00:00, 160MB/s]


In [5]:
messages_system= {"role": "system",
                        "content": '''You are a simulated IELTS Speaking examiner and i am a candidate of this exam. Your primary task is to conduct a mock IELTS Speaking test, adhering closely to the IELTS standards.
# The IELTS standards are as follows:
    Part 1: Introduction and Interview (4-5 minutes)
    - Introduce yourself and confirm the candidate's identity.
    - Ask questions about familiar topics such as home, family, work, studies, and interests.
    - Example questions:
      - Can you tell me about your hometown?
      - What do you enjoy doing in your free time?
      - Can you describe your job or what you are studying?

    Part 2: Long Turn (3-4 minutes)
    - Provide a task card with a topic and points to cover.
    - Give the candidate 1 minute to prepare and make notes if they wish.
    - Ask the candidate to speak for 1-2 minutes on the topic.
    - Example task card:
      - Describe a memorable trip you have taken.
        - Where you went?
        - Why you went there?
        - What you did there?
        - Explain why this trip was memorable for you?
    - After the candidate's long turn, ask one or two follow-up questions related to the topic.

    Part 3: Discussion (4-5 minutes)
    - Engage in a discussion based on the topic from Part 2.
    - Ask more abstract and complex questions to allow the candidate to express opinions and ideas.
    - Example questions:
      - How do you think travel experiences can change a person's view of the world?
      - What are the benefits and drawbacks of traveling to different countries?
      - How has tourism impacted the environment in popular travel destinations?
# Process
1. Ask the candidate if they want to:
    a. Simulate a mock IELTS Speaking test
    b. Analyze a transcript of an existing recorded IELTS Speaking test

2. Based on the candidate's choice:
    a. If they choose to simulate a mock IELTS Speaking test:
        - Conduct the test following the IELTS standards.
        - Structure the test into three parts of the IELTS standards.
        - Comply with the following regulations:
          - Display randomly topics for Part 1, and a single topic for Part 2 and 3, see more topics to choose from at page https://www.ielts-mentor.com/speaking.
          - Do not the reuse  example questions provided in the IELTS standards above.
          - please remember that you must ask the candidate question one by one it means that you must offer another question after the student give you response text which including prouncation scores, do not give questions one time because I do not want to respond it by one time
          - Focus on asking questions and responding appropriately without changing the candidate's words.
          - Maintain a strict adherence to the IELTS Speaking test format, avoiding off-topic conversations. If the user's input deviates, gently guide them back to the test format.
    b. If they choose to analyze a transcript of an IELTS Speaking test:
        - Classify the dialogue into examiner and test taker parts adhering closely to the IELTS standards .
        - Summarize the dialogue in your response.
        - Provide constructive feedback and next steps.

3. After the candidate answers the final question:
    - Provide constructive feedback based on their performance, highlighting strengths and areas for improvement.
    - Provide a strict estimate of specific overall band scores and sub-scores (e.g., if the sub-score is 6.0-6.5, your estimation should be 6.0) for fluency and coherence, lexical resource, grammatical range and accuracy, and pronunciation.
    - Include a disclaimer that this evaluation is generated by ChatGPT and is for reference only; official IELTS exam results may vary.
    then, ask the candidate if they want to a full, improved, and detailed answer based on the discussed topics and questions?
4. if the candidate do not want, end the discusion.
  If they want to a full, improved, and detailed answer based on the discussed topics and questions:
    - you make a full, improved, and detailed answer based on the discussed topics and questions in each part that used in this IELTS Speaking test.
    - Present questions and answers like:
          'Part number': Topic
            Question: Content Question
            Answer: Content your Answer
            ...
5. ask the candidate if they ant to a summary of the main challenges you faced during the simulation along with some key vocabulary and phrases to learn?
if they want:
    - summarize the main challenges, mistake the candidate faced during the simulation along.
    - then give some usefull vocabulary and phrases that came up during the test to learn.
'''
}
'''
'''

'\n'

In [ ]:

# Set your OpenAI API key
client = OpenAI(api_key = "sk-oGx8ZhLZ0xMahXqAF3tGcsTQhfO7WGIiRC0UlwtNYKJU9pxu",
                base_url="https://api.chatanywhere.tech/v1")
#mess = ""
#with open('prompt.txt', 'r') as file:
 ##   mess = file.read()
#messages_system= {"role": "system",
                 #      "content": mess
                   #    }

# Transcribe audio using OpenAI's Whisper
def transcribe_audio(audio):
    segment_length = 600000
    # Open the audio file
    audio_file = AudioSegment.from_file(audio)
    # Get the duration of the audio file in milliseconds
    duration_ms = len(audio_file)
    # Calculate the number of segments needed
    num_segments = math.ceil(duration_ms / segment_length)
    # Create an empty string to hold the concatenated text
    all_text = ""
    # Split the audio file into segments
    for i in range(num_segments):
        start = i * segment_length
        end = min((i + 1) * segment_length, duration_ms)
        segment = audio_file[start:end]
        segment.export(f"segment_{i}.mp3", format="mp3")

    for i in range(num_segments):
        audio_file = open(f"segment_{i}.mp3", "rb")
        transcript = W_model.transcribe(f"segment_{i}.mp3")
        all_text += transcript['text']
    return all_text
import csv
def respond(
    message,
    history: list[tuple[str, str]],
    mic=None,
    text = "Simulator IELTS Speaking test",
):
    global messages_system
    messages = [messages_system]
    audio = None
    if len(history) != 0:
      history_chat = []
      with open('chat_transcript.csv', 'r') as f:
        history_chat = csv.reader(f)
        for val in history_chat:
          if val[0]:
              messages.append({"role": val[0], "content": val[1]})

    if mic is not None:
          audio = mic
    elif  len(message["files"])!= 0:
          audio = message["files"][0]
          #return message["files"][0]
    #else:
          #return "You must either provide a mic recording or a file"
    transcript = ""
    if audio is not None:
        transcript = transcribe_audio(audio)
    transcript += "\n" + message["text"]

    messages.append({"role": "user", "content": transcript})

    response = ""
    response = client.chat.completions.create(
          messages= messages,
          model="gpt-3.5-turbo",
      )
    systems_message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": systems_message})
    chat_transcript =[]
    for message in messages:
        if message['role'] != 'system':
            chat_transcript.append([message['role'] , message['content']])
    with open('chat_transcript.csv', 'w') as f:
      writer = csv.writer(f)
      writer.writerows(chat_transcript)
    yield systems_message

"""
"""
demo = gr.ChatInterface(
    respond,
    additional_inputs=[
        gr.Audio(sources="microphone", type="numpy", label="Speak here..."),
    ],
    title="AI Auditor",
    description='''AI Alliance for Audio Analytics Team. Our project's objective is to conduct a mock IELTS Speaking test, adhering closely to the IELTS standards.
    upload a file record of your test or start a  ock IELTS Speaking test''',
    textbox = gr.MultimodalTextbox(interactive=True, file_types=["audio"], placeholder="upload file record..."),

    multimodal=True,
)


if __name__ == "__main__":
    demo.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2a7d569fa1df27ad89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Demo by gradio Interface

In [ ]:
# Set your OpenAI API key
client = OpenAI(api_key = "sk-oGx8ZhLZ0xMahXqAF3tGcsTQhfO7WGIiRC0UlwtNYKJU9pxu",
                base_url="https://api.chatanywhere.tech/v1")


# Transcribe audio using OpenAI's Whisper
def transcribe_audio(audio):
    segment_length = 600000
    # Open the audio file
    audio_file = AudioSegment.from_file(audio)
    # Get the duration of the audio file in milliseconds
    duration_ms = len(audio_file)
    # Calculate the number of segments needed
    num_segments = math.ceil(duration_ms / segment_length)
    # Create an empty string to hold the concatenated text
    all_text = ""
    # Split the audio file into segments
    for i in range(num_segments):
        start = i * segment_length
        end = min((i + 1) * segment_length, duration_ms)
        segment = audio_file[start:end]
        segment.export(f"segment_{i}.mp3", format="mp3")


    for i in range(num_segments):
        audio_file = open(f"segment_{i}.mp3", "rb")
        transcript = W_model.transcribe(f"segment_{i}.mp3")
        all_text += transcript['text']

    return all_text

# Analyze transcript using OpenAI's GPT-3
def analyze_transcript(transcript):
    global messages_system
    messages=[]
    messages.append(messages_system)
    messages.append({"role": "user", "content": transcript})
    response = client.chat.completions.create(
          messages= messages,
          model="gpt-3.5-turbo",
      )

    systems_message = response.choices[0].message.content
    messages.append({"role": "assistant", "content": systems_message})

    chat_transcript = ""
    for message in messages:
        if message['role'] != 'system':
            chat_transcript += message['role'] + ": " + message['content'] + "\n\n"
    result = "Transcript: " + transcript + "\n\n"
    return systems_message #+"\n\n"+ result
# Define the transcription and analysis function
def transcribe_and_analyze(mic=None, file=None):
    if mic is not None:
        audio = mic
    elif file is not None:
        audio = file
    else:
        return "You must either provide a mic recording or a file"
    # Transcribe the audio to text
    transcript = transcribe_audio(audio)
    # Analyze sentiment and summarize
    analysis = analyze_transcript(transcript)
    return analysis



In [ ]:

io = gr.Interface(fn=transcribe_and_analyze,
                      inputs=[
                          gr.Audio(sources="microphone", type="numpy", label="Speak here..."),
                          gr.Audio(sources="upload", type = "filepath"),
                      ],
                      outputs="text",
                      title="AI Auditor for",
                      description='''AI Alliance for Audio Analytics Team. Our project's objective is to conduct a mock IELTS Speaking test, adhering closely to the IELTS standards
                      ''',
                      examples=[[None, "./Samples/Sample1.mp3"],
                                [None, "./Samples/Sample2.mp3"],
                                [None, "./Samples/Sample3.mp3"]],
                      )

io.launch(share=True, debug = True)